In [36]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [37]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [38]:
x_train = torch.rand([42583, 1, 18, 300])
y_train = torch.ones([42583, 4]) 
y_train[:, :3] = 0
x_val = torch.rand([4731, 1, 18, 300])
y_val = torch.ones([4731, 4]) 
y_val[:, :3] = 0
x_test = torch.rand([11828, 1, 18, 300])
y_test = torch.ones([11828, 4]) 
y_test[:, :3] = 0

In [39]:
class MyDataset(Dataset):
    def __init__(self, x_train, x_test, y_train, y_test, train=True):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x_train if train else x_test
        self.y = y_train if train else y_test
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [40]:
class LSTM_CNN_Spatial(nn.Module):
    def __init__(self, num_classes, batch_size, T, C, input_size, hidden_size,
                 num_layers, spatial_num=8, drop_out=0.5):
        super(LSTM_CNN_Spatial, self).__init__()

        self.N = batch_size
        self.T = T
        self.C = C
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.pool = 4
        self.seq_len = self.T // self.input_size
        self.fc_in = spatial_num * self.hidden_size // self.pool
        
        self._lstm = nn.LSTM(self.input_size, self.hidden_size, 
                            self.num_layers, batch_first=True)
        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, spatial_num, (self.C, 1)),
            nn.BatchNorm2d(spatial_num),
            nn.ELU(),
            nn.AvgPool2d((1, self.pool)),
            nn.Dropout(drop_out)
        )
        
        self.fc = nn.Linear(self.fc_in , num_classes)
        
    def forward(self, x):
        # input shape of x: (N, 1, C, T)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.seq_len, self.input_size)
        _x = None
        for index, lstm in enumerate(self.lstm):
            lstm_out, _ = lstm(x[:, index, :, :], None)
            tmp = lstm_out[:, -1, :]
            tmp = tmp.unsqueeze(0)
            if _x is None:
                _x = tmp
            else:
                _x = torch.cat((_x, tmp), dim=0)
        
        # (C, N, H) ===> (N, 1, C, H)   H: hidden_size
        x = _x.permute(1, 0, 2).unsqueeze(1)
        x = self.block_1(x)
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return probas  

In [41]:
"""
if __name__ == '__main__':
    # model test
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    x = torch.randn((4, 1, 18, 300))
    x = x.to(DEVICE)
    # num_classes = 4, batch_size = 4, T = 256, C = 64, input_size = 16, hidden_size = 16, num_layers = 2, spatial_num=8, drop_out=0.5
    model = LSTM_CNN_Spatial(2, 4, 300, 18, 15, 160, 20)
    model = model.to(DEVICE)
    y = model(x)
    print(y.data)
"""

"\nif __name__ == '__main__':\n    # model test\n    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')\n    x = torch.randn((4, 1, 18, 300))\n    x = x.to(DEVICE)\n    # num_classes = 4, batch_size = 4, T = 256, C = 64, input_size = 16, hidden_size = 16, num_layers = 2, spatial_num=8, drop_out=0.5\n    model = LSTM_CNN_Spatial(2, 4, 300, 18, 15, 160, 20)\n    model = model.to(DEVICE)\n    y = model(x)\n    print(y.data)\n"

In [42]:
def train(model, criterion, optimizer, data_loader, device, train_num, epochs, logged=False):
    for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        batch_size = None
        for index, data in enumerate(data_loader):
            x, y = data
            batch_size = x.shape[0] if index == 0 else batch_size
            x, y = x.to(device), y.to(device)
            
            y_pred = model(x)
            _, pred = torch.max(y_pred, 1)
            
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
            
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += float(loss.item())
            
        batch_num = train_num // batch_size
        _loss = running_loss / (batch_num + 1)
        acc = correct_num / train_num * 100
        if not logged:
            print(f'Epoch {epoch+1}/{epochs}\tTrain loss: {_loss:.4f}\tTrain acc: {acc:.2f}%')
        
        print(model(x_train[0].cuda()))
    if not logged:
        print('Finish Training!')

In [43]:
def test(model, criterion, data_loader, device, test_num, log, logged):
    running_loss = 0.0
    correct_num = 0
    model.eval()
    batch_size = None
    for index, data in enumerate(data_loader):
        x, y = data
        batch_size = x.shape[0] if index == 0 else batch_size
        x, y = x.to(device), y.to(device)
        
        y_pred = model(x)
        _, pred = torch.max(y_pred, 1)
        if sys.platform == 'linux':
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
        else: 
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
        
        loss = criterion(y_pred, y)
        running_loss += float(loss.item())
    
    batch_num = test_num // batch_size
    _loss = running_loss / (batch_num + 1)
    acc = correct_num / test_num * 100
    print(f'Test loss: {_loss:.4f}\tTest acc: {acc:.2f}%')
    if logged:
        log.append(f'{acc:.2f}\t\n')
        with open('result.txt', 'a') as f:
            f.writelines(log)
    
    return model

In [48]:
def main(epochs, batch_size, input_size, hidden_size, num_layers, spatial_num, drop_out, logged=False):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('DEVICE: ', DEVICE)
    
    group = 1
    sorted_ = True
    # sorted_ = False

    # load data from '.npy' file
    # x_train, x_test, y_train, y_test = load_group_eeg_data(date, group, sorted_=sorted_)
    # x: (N, C, T)  N: trials  C: channels  T: times 
    train_num, test_num = x_train.shape[0], x_test.shape[0]
    
    # make dataset for train and test
    train_data = MyDataset(x_train, x_test, y_train, y_test)
    test_data = MyDataset(x_train, x_test, y_train, y_test, train=False)
    train_loader = DataLoader(train_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # model initiation
    # model = LSTM(num_classes=2, input_size=64, hidden_size=256, num_layers=2)
    
    # model = LSTM_CNN(num_classes=2, channels=x_train.shape[1], input_size=input_size, hidden_size=hidden_size, 
    #                  num_layers=num_layers, spatial_num=spatial_num, drop_out=drop_out)
    
    # model = LSTM_CNN_Half(num_classes=2, batch_size=batch_size, T=x_train.shape[-1],
    #                       C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
    #                       num_layers=num_layers, spatial_num=spatial_num)
    
    model = LSTM_CNN_Spatial(num_classes=4, batch_size=batch_size, T=x_train.shape[-1],
                          C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, spatial_num=spatial_num)
    
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)

    log = []
    if logged:
        log.append(f'{epochs}\t{batch_size}\t{input_size}\t{hidden_size}\t'
                   f'{num_layers}\t{spatial_num}\t{drop_out}\t')
    model2 = train(model, criterion, optimizer, train_loader, DEVICE,train_num, epochs, logged)
    test(model, criterion, test_loader, DEVICE, test_num, log, logged)

    return model2

In [49]:
epochs = 4
batch_size = 4096
input_size = 15
hidden_size = 16
num_layers = 4
spatial_num = 8
drop_out = 0.5
logged = False
model = main(epochs, batch_size, input_size, hidden_size, num_layers, spatial_num, drop_out, logged)

DEVICE:  cuda:0


  0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_8944\242123630.py:15: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
 25%|██▌       | 1/4 [00:02<00:07,  2.45s/it]

Epoch 1/4	Train loss: 1.3055	Train acc: 0.00%
tensor([[0.2205, 0.2366, 0.1078, 0.4351]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


 50%|█████     | 2/4 [00:04<00:04,  2.40s/it]

...

Epoch 3/4	Train loss: 1.0976	Train acc: 0.00%
tensor([[0.0762, 0.1582, 0.0952, 0.6704]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


100%|██████████| 4/4 [00:09<00:00,  2.42s/it]

Epoch 4/4	Train loss: 0.9793	Train acc: 0.00%
tensor([[0.1065, 0.0471, 0.0913, 0.7551]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
Finish Training!



C:\Users\admin\AppData\Local\Temp\ipykernel_8944\1666288419.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())


Test loss: 0.9641	Test acc: 0.00%


In [188]:
model = LSTM_CNN_Spatial(num_classes=4, batch_size=batch_size, T=x_train.shape[-1],
                          C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, spatial_num=spatial_num)

y_pred = model(x_train[2:50])
likeliness(y_pred, y_train[2:50])


0.10416666666666667

In [160]:
def likeliness(y_pred, y):
    pred = torch.zeros([len(y_pred), 4])
    for idx, unit in enumerate(torch.argmax(y_pred, axis = 1)):
        pred[idx, unit] = 1
    return accuracy_score(pred, y)